# Flexible Docking

Before you start:

1. Prepare the ligand using AutoDockTools.
2. Prepare the flexible (and rigid) receptor.
3. Prepare or define the search space.

In [1]:
from pathlib import Path

In [2]:
PROTEIN_CLS = "RAS"
PROTEIN_SID = "SM27"
LIGAND_NAME = "GDP"

WD = Path(f"wd/{PROTEIN_CLS}/{PROTEIN_SID}-{LIGAND_NAME}")
WD.mkdir(parents=True, exist_ok=True)

# You need to prepare the following:
PROTEIN_FLEX_PDBQT = WD / f"{PROTEIN_SID}.flex.pdbqt"
PROTEIN_RIGID_PDBQT = WD / f"{PROTEIN_SID}.rigid.pdbqt"
LIGAND_PDBQT = WD / f"{LIGAND_NAME}.pdbqt"
assert PROTEIN_FLEX_PDBQT.exists() and PROTEIN_RIGID_PDBQT.exists() and LIGAND_PDBQT.exists()

# And define the following search space:
SPACE_CONF = WD / "vina.conf"

# Docking settings:
NUM_MODES = 25
ENERGY_RANGE = 5
EXHAUSTIVENESS = 20

OUTPUT_PDBQT = WD / f"{PROTEIN_SID}-{LIGAND_NAME}.docked.pdbqt"

In [4]:
!cd {WD} && vina --receptor {PROTEIN_RIGID_PDBQT.name} --flex {PROTEIN_FLEX_PDBQT.name} --ligand {LIGAND_PDBQT.name} --config {SPACE_CONF.name} --out {OUTPUT_PDBQT.name} --num_modes {NUM_MODES} --energy_range {ENERGY_RANGE} --exhaustiveness {EXHAUSTIVENESS} | tee vina.log

#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, Journal of Computational Chemistry 31 (2010)  #
# 455-461                                                       #
#                                                               #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see http://vina.scripps.edu for more information.      #
#################################################################

Detected 24 CPUs
Reading input ... done.
Setting up the scoring function ... done.
Analyzing the binding site ... done.
Using random seed: 10

In [7]:
OUTPUT_LIGAND_PREFIX = f"{LIGAND_NAME}.docked."
OUTPUT_FLEX_PREFIX = f"{PROTEIN_SID}.docked.flex."

!cd {WD} && vina_split --input {OUTPUT_PDBQT.name} --ligand {OUTPUT_LIGAND_PREFIX} --flex {OUTPUT_FLEX_PREFIX}

In [15]:
from pymol import cmd

for ligand_pdbqt in WD.glob(f"{OUTPUT_LIGAND_PREFIX}*.pdbqt"):
    i = ligand_pdbqt.stem.split(".")[-1]
    flex_pdbqt = WD / f"{OUTPUT_FLEX_PREFIX}{i}.pdbqt"
    assert flex_pdbqt.exists()
    cmd.delete('all')
    cmd.reinitialize()
    cmd.load(PROTEIN_RIGID_PDBQT, "RIGID")
    cmd.load(flex_pdbqt, "FLEX")
    cmd.load(ligand_pdbqt, "LIGAND")
    cmd.create("COMPLEX", "RIGID or FLEX or LIGAND")
    cmd.save(f"{WD}/{PROTEIN_SID}-{ligand_pdbqt.stem}.pdb", "COMPLEX")